# Import libraries

In [1]:
import azureml.core
from azureml.core import Model, Workspace, Webservice

# Load Workspace and Web Service

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print("Ready to use Azure ML {} to work with {}".format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.0.83 to work with sbirkamlws


In [3]:
# Retrieve model trained by automl

service_name = 'aks-service-2'

service = Webservice(ws, service_name)

# Use Model with Azure ML SDK

In [4]:
import json

x_new = [["1111-TESTC","Female",0,1,0,28,1,1,"Fiber optic",0,0,1,1,1,1,"Month-to-month",1,"Electronic check",104.8,3046.05]]
print ('Customer with ID: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes)

Customer with ID: ['1111-TESTC', 'Female', 0, 1, 0, 28, 1, 1, 'Fiber optic', 0, 0, 1, 1, 1, 1, 'Month-to-month', 1, 'Electronic check', 104.8, 3046.05]
{'result': [False]}


# Use Model with HTTP request

In [5]:
# Get web service endpoint
endpoint = service.scoring_uri
print(endpoint)

http://52.240.157.60:80/api/v1/service/aks-service-2/score


In [6]:
# First try without authentication
import requests
import json

x_new = [["1111-TESTC","Female",0,1,0,28,1,1,"Fiber optic",0,0,1,1,1,1,"Month-to-month",1,"Electronic check",104.8,3046.05]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json', }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
# # if (key) auth is enabled, retrieve the API keys. AML generates two keys.
# key1, Key2 = aks_service.get_keys()
# print(key1)

# # if token auth is enabled, retrieve the token.
access_token, refresh_after = service.get_token()

In [8]:
# # If token auth is enabled, don't forget to add token to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

In [9]:
import requests
import json

x_new = [["1111-TESTC","Female",0,1,0,28,1,1,"Fiber optic",0,0,1,1,1,1,"Month-to-month",1,"Electronic check",104.8,3046.05]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# # If token auth is enabled, don't forget to add token to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes)

Patient ['1111-TESTC', 'Female', 0, 1, 0, 28, 1, 1, 'Fiber optic', 0, 0, 1, 1, 1, 1, 'Month-to-month', 1, 'Electronic check', 104.8, 3046.05] {'result': [False]}
